## COW Radar Plotting with py-ART

This code is based on py-ART tutorials available at https://arm-doe.github.io/pyart/examples/index.html but is modified slightly for use specifically with the Illinois FARM C-Band on Wheels (COW) radar.

In [ ]:
import pyart
import matplotlib.pyplot as plt

In [ ]:
# Read in radar data in cfradial format (this is an example of the COW's file formatting)
radar = pyart.io.read('cfrad.20220322_230026.916_to_20220322_230039.782_COW1high_SUR.nc')

In [ ]:
# List available radar products in this file
radar.fields.keys()

In [ ]:
# In order to plot data, pyART needs to know what type of scan the file contains (PPI vs. RHI).
# This may not be contained within the COW data files, so we can set it manually. In this case
# our data is in the form of a PPI scan.
radar.scan_type = 'ppi'

In [ ]:
# We can now create a basic plot of reflectivity
# We've also set a vmin and vmax, changed the colormap and added range rings, cross hairs,
# and grid lines.
display = pyart.graph.RadarDisplay(radar)
fig = plt.figure(figsize=(14,12))
ax = fig.add_subplot(111)
display.plot('DBZHC', 0, vmin=-32, vmax=64., cmap='pyart_Carbone42')
display.plot_range_rings([30, 60, 89])
display.plot_cross_hair(2.)
display.plot_grid_lines()

plt.show()

In [ ]:
# We can also create a multi-panel plot using standard matplotlib subplot conventions. Here 
# we are plotting reflectivity and velocty side-by-side.
display = pyart.graph.RadarDisplay(radar)
fig = plt.figure(figsize=(14,12))
ax = fig.add_subplot(221)
display.plot('DBZHC', 0, vmin=-32, vmax=64., cmap='pyart_Carbone42')
display.plot_range_rings([30, 60, 89])
display.plot_cross_hair(2.)
display.plot_grid_lines()

ax = fig.add_subplot(222)
display.plot('VEL', 0, vmin=-32, vmax=32., cmap='pyart_Carbone42')
display.plot_range_rings([30, 60, 89])
display.plot_cross_hair(2.)
display.plot_grid_lines()

plt.show()

In [ ]:
# Plotss can be zoomed into a particular area using display.set_limits and specifying
# x and y limits in kilometers from the radar.
display = pyart.graph.RadarDisplay(radar)
fig = plt.figure(figsize=(14,12))
ax = fig.add_subplot(111)
display.plot('DBZHC', 0, vmin=-32, vmax=64., cmap='pyart_Carbone42')
display.plot_range_rings([10, 20, 30])
display.plot_cross_hair(1.)
display.plot_grid_lines()
display.set_limits(xlim=(-15, 15), ylim=(-30, 0))

plt.show()

Py-ART also has a number of advanced data editing features that allow for quality control. Some detailed examples are available at this link (http://wiki.lrose.net/index.php/Csu_radartools_tutorial) and on the py-ART website within the pyart.correct section.

Here, we will only demonstrate the masking feature as it is frequently useful and simple to use.

In [ ]:
# Create a gatefilter object
gatefilter = pyart.correct.GateFilter(radar)

# Mask all regions (within all products) where reflectivity is below 5 dBZ
gatefilter.exclude_below('DBZHC', 5)

# Remove any speckled values that are left behind when these values are masked
pyart.correct.despeckle_field(radar, 'DBZHC', label_dict=None, threshold=- 100, size=10, 
                                gatefilter=gatefilter, delta=5.0)

# Setup our display
display = pyart.graph.RadarDisplay(radar)
fig = plt.figure(figsize=(14,12))
ax = fig.add_subplot(221)
display.plot('DBZHC', 0, vmin=-32, vmax=64., cmap='pyart_Carbone42', gatefilter=gatefilter)
display.plot_range_rings([30, 60, 89])
display.plot_cross_hair(2.)
display.plot_grid_lines()
#display.set_limits(xlim=(-15, 15), ylim=(-30, 0))

ax = fig.add_subplot(222)
display.plot('VEL', 0, vmin=-32, vmax=32., cmap='pyart_Carbone42', gatefilter=gatefilter)
display.plot_range_rings([30, 60, 89])
display.plot_cross_hair(2.)
display.plot_grid_lines()
#display.set_limits(xlim=(-15, 15), ylim=(-30, 0))

plt.show()